In [1]:
# problem configuration
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]
# algorithm configuration
max_iter = 100
max_candidates = 50
increase = 1.3
decrease = 0.9
# execute the algorithm
# best = search(berlin52, max_candidates, max_iter, increase, decrease)

In [4]:
import math
import random
from random import sample
import numpy as np

def euc_2d(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

def cost(sol,cities):
    distance = 0 
    for i,c1 in enumerate(sol):
        if (i == len(sol)-1):
            c2 = sol[0]
        else:
            c2 = sol[i+1]
        distance += euc_2d(cities[c1],cities[c2])
    return distance

def random_permutation(cities):
    perm = np.random.permutation(len(cities))
    return perm

def stochastic_2_op(x):
    new_x = []
    x = list(x)
    start,end = sorted(sample(range(2,len(x)),2))
    new_x = x[:start]
    new_x += (list(reversed(x[start:end])))
    new_x += x[end:]
    edges = [[start-1,start],[end-1,end]]
    return new_x,edges

def is_tabu(x,tabu_list):
    for i,c1 in enumerate(x):
        if i == len(x)-1:
            c2 = x[0]
        else:
            c2 = x[i+1]
        if [c1,c2] in tabu_list:
            return True
    return False

In [7]:
def make_tabu(tabu_list, edge, iter):
    for entry in talu_list:
        if entry['edge'] == edge:
            entry['iter'] = iter
            return entry
    entry = {'edge':edge,'iter':iter}
    tabu_list.append(entry)
    return entry

def to_edge_list(perm):
    li = []
    for i,c1 in enumerate(perm):
        if i == len(perm)-1:
            c2 = perm[0]
        else:
            c2 = perm[i+1]
        li.append([c1,c2])
    return li

In [9]:
def equivalent(el1, el2):
    ## 如果el1的item 都在el2中，
    if el1 in el2:
        return False
    else:
        return True

In [35]:
def generate_candidate(best, cities):
    candidate = {}
    candidate['vector'], edges = stochastic_two_opt(best['vector'])
    candidate['cost'] = cost(candidate['vector'], cities)
    candigate['edges'] = edges
    return candidate

In [36]:
def get_candidate_entry(visited_list, permutation):
    edgeList = to_edge_list(permutation)
    for entry in visited_list:
        if equivalent(edgeList,entry['edgelist']):
            return entry
        else:
            return False

def store_permutation(visited_list, permutation, iteration):
    entry = {}
    entry['edgelist'] = to_edge_list(permutation)
    entry['iter'] = iteration
    entry['visits'] = 1
    visited_list.append(entry)
    return entry

In [37]:
def sort_neighborhood(candidates, tabu_list, prohib_period, iteration):
    tabu, admissable = [], []
    for a in candidates:
        if is_tabu(a[1][0], tabu_list, iteration, prohib_period) or is_tabu(a[1][1], tabu_list, iteration, prohib_period):
            tabu.append(a)
        else:
            admissable << a
    return [tabu, admissable]

In [38]:
def search(cities, max_cand, max_iter, increase, decrease):
    current = {'vector':random_permutation(cities)}
    current['cost'] = cost(current['vector'],cities)
    best = current
    tabu_list,prohib_period = [],1
    visited_list, avg_size, last_change = [], 1, 0
    for it in range(max_iter):
        candidate_entry = get_candidate_entry(visited_list, current['vector'])
        if not candidate_entry :
            repetition_interval = it - candidate_entry['iter']
            candidate_entry['iter'] = it
            candidate_entry['visits'] += 1
            if repetition_interval < 2* (len(cities)-1):
                avg_size = 0.1*(iter-candidate_entry['iter']) + 0.9*avg_size
                prohib_period = (prohib_period.to_f * increase)
                last_change = iter
            else:
                store_permutation(visited_list, current['vector'], iter)
        if it-last_change > avg_size:
            prohib_period = [prohib_period*decrease,1].max
            last_change = it
        candidates = [generate_candidate(current,cities) for i in range(max_cand)]
        candidates.sort(key = lambda k:k['cost'])
        tabu,admis = sort_neighborhood(candidates,tabu_list,prohib_period,iter)
        if len(admis) < 2:
            prohib_period = len(cities)-2
            last_change = it
        if admis is None:
            current,best_move_edges = tabu[0]
        else:
            current,best_move_edges = admis[0]
        for edge in best_move_edges:
            make_tabu(tabu_list,edge,it)
        if candidates[0]['cost'] < best['cost']:
            best = candidates[0]
        return best['vector'],best['cost']

In [39]:
search(berlin52, max_candidates, max_iter, increase, decrease)

TypeError: 'NoneType' object is not subscriptable